# Calling genotypes, prevalences and filtering data
This program analyzes tables of counts. In the prior "variant calling" step of the program, mutation counts were generated for each sample by annotating the VCF file generated by freebayes, using SNPEff to assign amino acid names to mutations.

The count tables are intended to be as "raw" as possible. In this part of the analysis we will:
1. apply various filters to the count tables, considering a mutation 'sampled' for a sample if it passes the filters for that sample.
2. choose the level of detail of geographic locations we care about from a metadata file
3. summarize the number of filtered samples at each location that had the mutation and the total number of filtered samples at each location, where the 'prevalence' of a mutation at a location is the first number divided by the second number
4. graph the prevalences and filtered sample sizes on a map
5. (optional) - apply additional filtering for re-evaluation of prevalence data.

In [ ]:
# RUN
import sys
sys.path.append("/opt/src")
import calculate_prevalences as cap
import os
import pandas as pd
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
wdir = "/opt/user/stats_and_variant_calling/"
import ipywidgets as widgets
import yaml

In [ ]:
# RUN

mutation_count_file = "alternate_AA_table.csv"
mutation_coverage_file = "coverage_AA_table.csv"

In [ ]:
# RUN
import pandas as pd
mutation_counts = pd.read_csv(mutation_count_file,
                              header=list(range(6)),
                              index_col=0)
mutation_coverage = pd.read_csv(mutation_coverage_file,
                                index_col=0,
                                header=list(range(6)))

metadata_table=pd.read_csv('/opt/user/prevalence_metadata/metadata.csv')

mutation_handle=open(mutation_count_file)
mutation_handle.readline()
mutation_handle.readline()
mutations=mutation_handle.readline().strip().split(',')
filtered_mutations=[]
for mutation in mutations[1:]:
    change=mutation.split('-')[-1]
    if 'ins' not in change and 'del' not in change and 'dup' not in change:
        filtered_mutations.append(mutation)
print('observed mutations are:\n\n')
print(filtered_mutations)
print('\n\nmetadata table is')
metadata_table.head()

In [ ]:
# USER INPUT

# how many reads (or, more accurately, UMIs) do we need spanning the genomic coordinates of a mutation in order to
# consider the coordinates "sequenced" for a given sample? If UMI count is below min_coverage, the sample will be
# marked as "missing" for the mutation of interest.
min_coverage = 10

# If min_coverage is achieved in the sample, how many UMIs do we need that support the mutant (or alternate) allele
# in order to consider the alternate allele "present" in the sample? Otherwise if min_coverage is surpassed but
# min_count is not, the sample will be marked as matching the reference allele.
min_count = 3

# Is there a number that min_count/min_coverage needs to surpass before we consider an alternate allele "present" in a
# sample? Our usual answer is 'no' so we generally set this to '0' - but you may not want to count an alternate allele
# present if it's found in a very low fraction of the UMIs associated with a sample, even if the absolute number of UMIs
# associated with the alternate allele is above your threshold.
min_freq = 0

# Which mutations would you like to graph? A full list of sequenced plus key mutations is created when you run the code in the
# cell above.

#example if you want only "key mutations from the targets.tsv file found in the project_resources folder"
mutation_column=open('/opt/project_resources/targets.tsv').readline().strip().split('\t').index('mutation_name')
mutations_of_interest=[line.strip().split('\t')[mutation_column] for line in open('/opt/project_resources/targets.tsv')][1:]

#example if you want only mutations from the AA tables from a specific gene
#mutations_of_interest=[mutation for mutation in filtered_mutations if 'k13' in mutation]

#example if you want to manually set the mutations to analyze
#mutations_of_interest=[
#'dhfr-ts-Asn51Ile', 'dhfr-ts-Cys59Arg', 'dhfr-ts-Ser108Asn', 'dhfr-ts-Ile164Leu',
#'mdr1-Asn652Asp', 'mdr1-Asn86Tyr', 'mdr1-Asp1246Tyr', 'mdr1-Asp650Asn',
#'mdr1-Tyr184Phe', 'crt-Met74Ile', 'crt-Asn75Glu', 'crt-Lys76Thr',
#'crt-Gln271Glu', 'crt-Arg371Ile', 'dhps-Ala437Gly', 'dhps-Lys540Glu',
#'dhps-Ala581Gly']


# In the metadata sheet (reproduced in the table output from the cell above), which column contains sample names and
# which column contains the names of the locations you'd like to plot (or summarize by)?
sample_column='Sampleid'
summary_column='Sites'

In [ ]:
# Which country would you like to graph?
country_shortcuts={'DRC': 
                   {"lat": -1.6815695315287824,
                    "lon": 22.744896416745945,
                    'zoom': 3.8},
                   'Uganda': 
                   {"lat": 1.5,
                    'lon': 32,
                    'zoom': 5.6},
                   'Tanzania':
                   {"lat": -5.7,
                    "lon": 35,
                    'zoom': 5.2},
                  }

country = widgets.Dropdown(
    options=country_shortcuts.keys(),
    description='Country:',
    disabled=False,
)
print("Country to zoom in and center the map on:\n")
display(country)



In [ ]:
# RUN
country=country.value
zoom_level=country_shortcuts[country]['zoom']
latitude=country_shortcuts[country]['lat']
longitude=country_shortcuts[country]['lon']
# OPTIONAL USER INPUT:
# You can override the values above and set a custom zoom level and centering below by commenting these lines in
#zoom_level=4.8
#latitude=1.5
#longitude=32

In [ ]:
# RUN

# import the PCA module which has genotype calling and
# filtering functions
import PCA

gt_calls = PCA.call_genotypes(mutation_counts, mutation_coverage,
                              min_count, min_coverage, min_freq)
#print('options are', gt_calls.keys())

# This step outputs samples that surpass the coverage threshold
# as their original coverage value and resets samples not passing the threshold as '0'.
# Note that even if there is insufficient coverage the resulting cell will be a '0' and not NaN
filtered_coverage=gt_calls["filtered_mutation_coverage"]
filtered_coverage.to_csv(os.path.join(wdir, "filtered_coverage_AA_table.csv"))

# This step outputs samples containing alternate alleles that surpass the coverage and alternate thresholds
# as their original values and samples not passing both thresholds as '0'. Note that even if there is insufficient
# coverage to make a call the output for this table will still be listed as '0'
filtered_mutation_counts = gt_calls["filtered_mutation_counts"]
filtered_mutation_counts.to_csv(os.path.join(
        wdir, "filtered_alternate_AA_table.csv"))

# This step outputs the within sample allele frequencies of each alternate allele. Unlike the tables above, this
# one displays NaN if the coverage levels do not surpass the threshold (due to dividing alternate counts by 0)
freq = gt_calls["wsaf"]
freq.to_csv(os.path.join(
        wdir, "within_sample_allele_frequencies.csv"))
freq.head()

# This step outputs a genotypes table, using the wsaf table as input. Cells can be NaN (coverage is below the
# threshold), 0 (frequency of the mutation is less than or equal to min_freq), 1 (frequency is bigger than min_freq but
# less than 100%) or 2 (mutation is found in all of the UMIs for the sample.
genotypes = gt_calls["genotypes"]
genotypes.to_csv(os.path.join(
        wdir, "filtered_genotypes_table.csv"))


# The outputs of this table are the same as the genotypes table, except this table does not differentiate between
# alternate alleles that are found in all UMIs of a sample and those that are found in some of the UMIs
# (both '2' and '1' values from the genotypes table get reset to '1')
prevalences_input = gt_calls["prevalences"]
prevalences_input.to_csv(os.path.join(wdir, "prevalences_input_table.csv"))


# To view the outputs of any given step, comment in the corresponding table below (or open the corresponding csv
# file in a spreadsheet):
# filtered_coverage.head()
# filtered_mutation_counts.head()
# genotypes.head()
# prevalences_input.head()

# Calculate Prevalence in Each Region

In [ ]:
# RUN
# Intersects the variants of interest with the table header
# default sort will sort mutations by gene and then reference amino acid name - this sorter sorts by gene and
# then amino acid position
def special_sort(mutations):
    sorting_list=[]
    for mutation in mutations:
        gene='-'.join(mutation.split('-')[:-1])
        pos=int(mutation.split('-')[-1][3:-3])
        sorting_list.append([gene, pos, mutation])
    return [item[-1] for item in sorted(sorting_list)]

prevalences_input='prevalences_input_table.csv'
variant_set = set(open(prevalences_input).readlines()[2].strip().split(',')[1:])
mutations_of_interest_set = set(mutations_of_interest)
variants_of_interest = list(variant_set.intersection(mutations_of_interest_set))
variants_of_interest=special_sort(mutations_of_interest)
print(variants_of_interest)

In [ ]:
# RUN
# This block loads the config file to find the associated metadata for your sample set. It summarizes the dataset by
# the user-specified summary_column and counts the number of '1' and '0' values from the prevalences_input_table
# for each named location found in the summary column. e.g. if there are 5 regions in a country and we're summarizing
# by region, there will be 5 named locations in the output table, and we will count the number of 'covered' samples in
# each region for each mutation along with the number of samples with the alternate allele for each mutation. These
# values appear as numerators and denominators in the output table below. Coordinates are averaged across all samples
# that share the same named location in the metadata. All sample sets need to share a single metadata file - this is to
# avoid naming conflicts in column names between different sample sets.
config_file = open('/opt/config/config_v0.5.1.yaml')
config_dict = yaml.safe_load(config_file)

prevalences_input_table = 'prevalences_input_table.csv'

contents=os.listdir('/opt/user/prevalence_metadata')
print('contents are', contents)

# get the sample sets from the user defined config file
sample_sets = config_dict['sample_set'].split(',')
if '' in sample_sets:
    sample_sets.remove('')
sample_sets = [x.replace(' ','') for x in sample_sets]

#path to the metadata - this is currently hardcoded
metadata_file = '/opt/user/prevalence_metadata/metadata.csv'

# generate a prevalence table for each set of samples
output_summary_table = 'prevalence_summary.tsv'

#    metadata_file='merged_corrected_renamed_samples_metadata.csv'
cap.calculate_prevalences(metadata_file,
                          prevalences_input_table,
                          mutations_of_interest,
                          output_summary_table, sample_column, summary_column)
prevalences=pd.read_csv(output_summary_table,
                          header=list(range(1)),
                          index_col=0, sep='\t')
prevalences.head(10)

# Plot Scatter map

In [ ]:
# RUN
import math
import numpy as np
def make_detail_graph(variant, summary_column):
    df = pd.read_csv("prevalence_summary.tsv", sep='\t')
    if variant in list(df)[3:]:
        df["prevalence"] = df[variant].str.split(" ").str[0].astype(float)
        #df["prevalence_percent"] = df["prevalence"]*100+1
        #df["prevalence_log"]=np.log2(df["prevalence_percent"])
        max_prevalence = max(df["prevalence"].to_list())
        #max_prevalence=7
        #max_prevalence=1.0
        df["sample_size"] = (
            df[variant].str.split("/").str[1].str.replace(")", "").astype(float)
        )
        df = df[df["sample_size"] > 0]
        # df = df[df["Dataset"] == int(dataset)]

        fig = px.scatter_map(
            df,
            lat="Latitude",
            lon="Longitude",
            #color="prevalence_log",
            color="prevalence",
            size="sample_size",
            size_max=50,
            # color_continuous_scale='cividis',
            range_color=(0, max_prevalence), # use this to make the most prevalent mutation "pop" as bright yellow
            zoom=zoom_level,
            hover_name=summary_column,
            height=800,
            width=800,
            hover_data=["sample_size"],
            center={"lat": latitude, "lon": longitude},
            title=variant+'_'+summary_column+'_0-'+str(max_prevalence),
        )
    fig.update_layout(
        mapbox=dict(
            layers=[
                dict(
                    sourcetype="geojson",
                    source={"type": "FeatureCollection", "features": []},
                    type="fill",
                    color="blue",
                    below="traces",
                    visible=False  # Hide the layer by default
                    )
    ]
    )
    )
    return fig

In [ ]:
# RUN
variants_of_interest=open("prevalence_summary.tsv").readline().split('\t')[3:]
variant = widgets.Dropdown(
    options=variants_of_interest,
    description='variant:',
    disabled=False,
)
display(variant)

In [ ]:
fig=make_detail_graph(variant.value, summary_column)
fig.show()
title_string=f'{variant.value}_{summary_column}'
#fig.write_image(title_string+'.svg')
#fig.write_image(title_string+'.png')
#fig.write_html(title_string+'.html')

In [ ]:
# Google AI suggested version (output below) - this version is required if you want to hide rivers, minor cities,
# major roads, district boundaries, etc. from the map - but this comes at the expense of loss of country labels.

# This code is still not fully implemented. If we went this route, we would need a box for user to manually add
# country names and GPS coordinates for each. A full implementation would also add a scale bar, country zoom dropdown
# menu, and colored dots like the ones above
import plotly.graph_objects as go

# Sample data - currently set to some arbitrary points in the US, easily reconfigurable
data = {'lat': [40.7128, 34.0522, 41.8781], 'lon': [-74.0060, -118.2437, -87.6298]}

fig = go.Figure(go.Scattergeo(
    lat=data['lat'],
    lon=data['lon'],
    mode='markers',
    marker=dict(size=10)
))

fig.update_layout(
    geo=dict(
        scope='africa',
        showland=True,
        landcolor='rgb(217, 217, 217)',
        showcountries=True,
        countrycolor='rgb(255, 255, 255)',
        showcoastlines=True,
        coastlinecolor='rgb(128, 128, 128)',
        showframe=False,  # Remove the outer frame
        showlakes=True,
        lakecolor='rgb(173, 216, 230)',
        showocean=True,
        oceancolor='rgb(128, 177, 211)',
        projection_type='mercator',  # Use a natural earth projection
    ),
    margin=dict(l=0, r=0, t=0, b=0)  # Remove margins
)

fig.update_layout(
    autosize=True,
    height=800,
    geo=dict(
        center=dict(
            lat=latitude,
            lon=longitude
        ),
        scope='africa',
        projection_scale=6.3
    )
)
fig.show()

In [ ]:
# RUN
# This code generates a bar chart for your mutations of interest
def get_prevalences(prevalence_file, mutation_list):
	prevalence_dict={'mutations':[], 'prevalences':[], 'gene_name':[]}
	h_dict={}
	for line_number, line in enumerate(open(prevalence_file)):
		line=line.strip().split('\t')
		if line_number==0:
			for column_number, column in enumerate(line):
				h_dict[column]=column_number
		elif line[0]=='overall':
			for mutation in mutation_list:
				if mutation in h_dict:
					column=h_dict[mutation]
					prevalence=float(line[column].split(' ')[0])
					prevalence_dict['mutations'].append(mutation)
					prevalence_dict['prevalences'].append(prevalence)
					prevalence_dict['gene_name'].append(mutation.split('-')[0])
				else:
					print(mutation, f'is not found in {prevalence_file}!!')
	return prevalence_dict

def make_bar_chart(prevalence_dict, graph_type="bar chart of variants_of_interest", y_max=1.0):
	import pandas as pd
	import plotly.express as px
	df = pd.DataFrame(prevalence_dict)
	fig1=px.bar(df, x='mutations', y='prevalences', color='gene_name', text_auto=False)
	fig1['layout']['xaxis']['title']=graph_type
	fig1['layout']['yaxis']['title']='overall prevalence'
	fig1.update_layout(yaxis=dict(range=[0, y_max]))
	fig1.write_html(f'{graph_type}.html')
	fig1.write_image(f'{graph_type}.svg')
	fig1.write_image(f'{graph_type}.png')
	return fig1
chart_name='drug resistance mutations'
y_max=1.0

In [ ]:
# OPTIONAL USER INPUT
# User can optionally reset the variants_of_interest to graph, chart_name, and y_max values (e.g. by commenting in one
#of the blocks of lines below), otherwise the values will be pulled from the cells above

#variants_of_interest=['crt-Met74Ile', 'crt-Asn75Glu', 'crt-Lys76Thr','crt-Gln271Glu',
#'crt-Arg371Ile', 'dhfr-ts-Asn51Ile', 'dhfr-ts-Cys59Arg', 'dhfr-ts-Ser108Asn',
#'dhfr-ts-Ile164Leu', 'dhps-Ala437Gly', 'dhps-Lys540Glu', 'dhps-Ala581Gly',
#'mdr1-Asn86Tyr', 'mdr1-Tyr184Phe', 'mdr1-Asp650Asn', 'mdr1-Asn652Asp',
#'mdr1-Asp1246Tyr']
#chart_name='non-k13 mutations'
#y_max=1.0

#variants_of_interest=['k13-Lys108Glu', 'k13-His136Asn', 'k13-Lys189Asn', 'k13-Lys189Thr',
#'k13-Arg255Lys', 'k13-Leu258Met', 'k13-Gln271His', 'k13-Cys469Phe',
#'k13-Cys469Tyr', 'k13-Asn490Thr', 'k13-Arg561His', 'k13-Ala578Ser',
#'k13-Ala675Val']
#chart_name='k13 mutations'
#y_max=0.5


# NO USER INPUT BELOW
prevalence_dict=get_prevalences('prevalence_summary.tsv', variants_of_interest)
fig1=make_bar_chart(prevalence_dict, chart_name, y_max)
fig1.show()

## Filter genotypes / prevalences
It is generally a good idea to do some basic noise removal once the genotypes are created. Some suggestions are provided here.

### Filter variants that are always at low WSAF
If a variant is only seen at a low frequency within samples, it is a good indication that it could be just noise. Here we will set a number of samples and minimum WSAF threshold to remove such noise.

```python
num_samples_wsaf = 2
min_wsaf = 0.5
wsaf_filter = ((freq > min_wsaf).sum()) >= num_samples_wsaf
```

The above options will keep the variants that are in at > 0.5 WSAF in at least 2 samples.

In [ ]:
# USER INPUT
num_samples_wsaf = 2
min_wsaf = 0.5

In [ ]:
wsaf_filter = ((freq > min_wsaf).sum()) >= num_samples_wsaf
print(("{} of {} variants will remain after the wsaf filter").format(
    wsaf_filter.sum(), freq.shape[1]))

### Filter variants that are observed with low UMI counts
If a variant is only supported by a low number of UMIs across the entire sample set, it is another indication of noise.

```python
num_samples_umi = 2
min_umi = 3
umi_filter = ((filtered_mutation_counts >= min_umi).sum()) > num_samples_umi
```

The above options will keep the variants that are supported by at least 3 UMIs in at least 2 samples.

In [ ]:
# USER INPUT
num_samples_umi = 2
min_umi = 3

In [ ]:
# RUN
umi_filter = ((filtered_mutation_counts >= min_umi).sum()) > num_samples_umi
print(("{} of {} variants will remain after the UMI filter").format(
    umi_filter.sum(), freq.shape[1]))

### Keep variants that were targeted
In most projects there are a number of variants that we would like to report, even if they are not seen in the sample set. We would like to stop those variants from being removed by the above filters.

In [ ]:
# RUN
targ = freq.columns.get_level_values("Targeted") == "Yes"

### Combine filters
Keep the variants that are either targeted or passing filters

In [ ]:
variant_mask = targ | (wsaf_filter & umi_filter)
print(("{} variants will remain in the final call set.\n"
       "{} variants were targeted and will be kept; and {} will be removed by "
       "the combined UMI and WSAF filters.").format(
    variant_mask.sum(), targ.sum(), (wsaf_filter & umi_filter).sum()))

## Filter data tables with the combined filters

In [ ]:
filtered_genotypes = genotypes.loc[:, variant_mask]
filtered_genotypes.to_csv(os.path.join(wdir, "final_filtered_genotypes.csv"))
filtered_genotypes.head()

In [ ]:
filtered_prevalences = prevalences.loc[:, variant_mask]
filtered_prevalences.to_csv(os.path.join(wdir, "final_filtered_prevalences_input_table.csv"))
filtered_prevalences.head()